In [6]:
from carla import Benchmark
import carla.evaluation.catalog as evaluation_catalog
from carla.data.catalog import OnlineCatalog
from carla.models.catalog import MLModelCatalog
from carla.models.negative_instances import predict_negative_instances
import carla.recourse_methods.catalog as recourse_catalog
import shap 
from sklearn.preprocessing import OrdinalEncoder
from carla.data.causal_model import CausalModel
import shap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from carla.data.catalog import CsvCatalog
import numpy as np 

import warnings
warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[autoreload of carla.data.load_scm.load_scm failed: Traceback (most recent call last):
  File "/home/jacqueline/.local/share/virtualenvs/CARLA-koH0yuP4/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/jacqueline/.local/share/virtualenvs/CARLA-koH0yuP4/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/home/jacqueline/.local/share/virtualenvs/CARLA-koH0yuP4/lib/python3.7/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/home/jacqueline/.local/share/virtualenvs/CARLA-koH0yuP4/lib/python3.7/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 630, in _exec
  File "<frozen importlib._bootstrap_external>", line 728, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/media/jacqueline/

In [7]:
import pandas as pd
# Load Data 
dataframe = pd.read_csv('./census.csv', index_col=['Unnamed: 0'])
continuous = dataframe.drop(columns=['target']).columns

dataset = CsvCatalog(file_path="census.csv",
                     continuous=continuous,
                     categorical=[],
                     immutables=[],
                     target='target')

In [8]:
#Model 

training_params = {"lr": 0.01, "epochs": 10, "batch_size": 16, "hidden_size": [18, 9, 2]}

ml_model = MLModelCatalog(
    dataset, model_type="ann", load_online=False, backend="pytorch"
)
ml_model.train(
    learning_rate=training_params["lr"],
    epochs=training_params["epochs"],
    batch_size=training_params["batch_size"],
    hidden_size=training_params["hidden_size"],
    force_train=True
)


balance on test set 0.24074529074529075, balance on test set 0.24100233386561848
Epoch 0/9
----------
train Loss: 0.3942 Acc: 0.8144

test Loss: 0.3583 Acc: 0.8380

Epoch 1/9
----------
train Loss: 0.3531 Acc: 0.8364

test Loss: 0.3815 Acc: 0.8220

Epoch 2/9
----------
train Loss: 0.3462 Acc: 0.8395

test Loss: 0.3641 Acc: 0.8349

Epoch 3/9
----------
train Loss: 0.3423 Acc: 0.8401

test Loss: 0.3500 Acc: 0.8329

Epoch 4/9
----------
train Loss: 0.3402 Acc: 0.8399

test Loss: 0.3304 Acc: 0.8423

Epoch 5/9
----------
train Loss: 0.3391 Acc: 0.8408

test Loss: 0.3366 Acc: 0.8424

Epoch 6/9
----------
train Loss: 0.3371 Acc: 0.8427

test Loss: 0.3617 Acc: 0.8294

Epoch 7/9
----------
train Loss: 0.3375 Acc: 0.8428

test Loss: 0.3837 Acc: 0.8209

Epoch 8/9
----------
train Loss: 0.3350 Acc: 0.8443

test Loss: 0.3402 Acc: 0.8368

Epoch 9/9
----------
train Loss: 0.3345 Acc: 0.8448

test Loss: 0.3422 Acc: 0.8354



In [9]:
from carla.models.negative_instances import predict_negative_instances
import carla.recourse_methods.catalog as recourse_catalog

factuals = predict_negative_instances(ml_model, dataset.df)
test_factual = factuals.iloc[:20]

display(test_factual)

,Unnamed: 0,Age,Workclass,Education-Num,Marital Status,...,Capital Gain,Capital Loss,Hours per week,Country,target
0,0,0.301370,0.875,0.800000,0.666667,...,0.02174,0.000000,0.397959,0.951220,False
1,1,0.452055,0.750,0.800000,0.333333,...,0.00000,0.000000,0.122449,0.951220,False
2,2,0.287671,0.500,0.533333,0.000000,...,0.00000,0.000000,0.397959,0.951220,False
3,3,0.493151,0.500,0.400000,0.333333,...,0.00000,0.000000,0.397959,0.951220,False
6,6,0.438356,0.500,0.266667,0.500000,...,0.00000,0.000000,0.153061,0.560976,False
7,7,0.479452,0.750,0.533333,0.333333,...,0.00000,0.000000,0.448980,0.951220,True
10,10,0.273973,0.500,0.600000,0.333333,...,0.00000,0.000000,0.806122,0.951220,True
11,11,0.178082,0.875,0.800000,0.333333,...,0.00000,0.000000,0.397959,0.463415,True
12,12,0.082192,0.500,0.800000,0.666667,...,0.00000,0.000000,0.295918,0.951220,False
13,13,0.205479,0.500,0.733333,0.666667,...,0.00000,0.000000,0.500000,0.951220,False


In [10]:
hyperparams = {"loss_type": "BCE", "binary_cat_features": True}

recourse_method = recourse_catalog.Wachter(ml_model, hyperparams)
df_cfs = recourse_method.get_counterfactuals(test_factual)

display(df_cfs)

[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactua

,Age,Capital Gain,Capital Loss,Country,Education-Num,...,Occupation,Race,Relationship,Sex,Workclass
0,0.321383,0.041736,0.020010,0.931264,0.820010,...,0.091441,1.019466,0.180003,0.980048,0.855055
1,0.442055,0.010000,0.010000,0.961219,0.810000,...,0.295714,1.010000,0.010000,1.010000,0.760000
2,0.344147,0.068804,0.066081,0.888660,0.601077,...,0.388635,0.933177,0.131541,1.069628,0.435270
3,0.445956,0.048099,0.047811,0.959460,0.448255,...,0.448365,0.546371,0.032423,0.999924,0.525846
6,0.409715,0.116150,0.116906,0.479262,0.384873,...,0.506366,0.618853,0.312817,0.119942,0.427526
7,0.499383,0.019938,0.019935,0.931477,0.553270,...,0.265813,1.019851,-0.019919,1.019861,0.730073
10,0.283973,0.010000,0.010000,0.941220,0.610000,...,0.275714,0.510000,-0.010000,1.010000,0.490000
11,0.188082,0.010000,0.010000,0.473415,0.810000,...,0.704286,0.260000,-0.010000,0.990000,0.865000
12,0.130829,0.049321,0.049459,0.956253,0.849467,...,0.113195,1.047949,0.646253,0.049377,0.544788
13,0.243899,0.039352,0.039515,0.944484,0.772818,...,0.821857,0.539116,0.160539,0.965284,0.460166


In [11]:
mapping_dict={
    "u1": "Sex",
    "u2": "Race",
    "u3": "Age",
    "u4": "Relationship",
    "u5": "Hours per week",
    "u6": "Capital Gain",
    "u7": "Education-Num",
    "u8": "Occupation",
    "u9": "target",}


In [14]:
def get_pred_from_causal(scm,values,cf_label, mapping_dict):
    #TODO change everything hard coded
    values['target']=cf_label
    def _get_noise_string(node):
        def _get_node_id(node):
            return node[1:]
        if not node[0] == "x":
            raise ValueError
        return "u" + _get_node_id(node)
    exogenous_variables = np.concatenate(
        [
            np.array(values[mapping_dict[node]]).reshape(-1, 1)
            for node in scm.get_topological_ordering("exogenous")
        ],
        axis=1,
    )
    exogenous_variables = pd.DataFrame(
        exogenous_variables, columns=scm.get_topological_ordering("exogenous")
    )

    endogenous_variables = exogenous_variables.copy()
    endogenous_variables = endogenous_variables.rename(
        columns=dict(
            zip(
                scm.get_topological_ordering("exogenous"),
                scm.get_topological_ordering("endogenous"),
            )
        )
    )
    # used later to make sure parents are populated when computing children
    endogenous_variables.loc[:] = np.nan
    for node in scm.get_topological_ordering("endogenous"):
        parents = scm.get_parents(node)
        if endogenous_variables.loc[:, list(parents)].isnull().values.any():
            raise ValueError(
                "parents in endogenous_variables should already be occupied"
            )
        endogenous_variables[node] = scm.structural_equations_np[node](
            exogenous_variables[_get_noise_string(node)],
            *[endogenous_variables[p] for p in parents],
        )
    labels=endogenous_variables['x9'][0]
    return labels

In [15]:
from carla.data import causal_model
from carla.evaluation import remove_nans
from carla.evaluation.api import Evaluation
class Sematic(Evaluation):
    """
    """

    def __init__(self, ml_model, causal_graph):
        self.ml_model= ml_model
        self.causal_graph=causal_graph
    def get_evaluation(self,factual: np.ndarray, counterfactual: np.ndarray, causal_model):
        # generate data 
        cf_label=self.ml_model.predict(np.array(counterfactual.values).reshape(1,-1))
        cf_label=np.argmax(cf_label)
        causal_label=get_pred_from_causal(self.causal_graph,counterfactual,cf_label, mapping_dict)
        print(cf_label)
        if cf_label ==causal_label:
            return 1 
        else: 
            return 0

class path_checker():
    pass

scm = CausalModel("adult")
metric = Sematic(ml_model,scm)
result=metric.get_evaluation(np.array([]),df_cfs.iloc[0],mapping_dict)
''' 
Analysis
Probitlity Distributions 
Histogram True Positive, False Positives 
'''
'''
Parts of counterfactuals actually change 
1hop and 2 hop relations 
'''
'''Histogram Features / true Positive / false Positive'''
# first initialize the benchmarking class by passing
# black-box-model, recourse method, and factuals into it
#benchmark = Benchmark(ml_model, recourse_method, factuals)

# now you can decide if you want to run all measurements
# or just specific ones.
#evaluation_measures = [
#    evaluation_catalog.YNN(benchmark.mlmodel, {"y": 5, "cf_label": 1}),
#    evaluation_catalog.Distance(benchmark.mlmodel),
#    evaluation_catalog.SuccessRate(),
#    evaluation_catalog.Redundancy(benchmark.mlmodel, {"cf_label": 1}),
#    evaluation_catalog.ConstraintViolation(benchmark.mlmodel),
#    evaluation_catalog.AvgTime({"time": benchmark.timer}),
#]

# now run all implemented measurements and create a
# DataFrame which consists of all results
#results = benchmark.run_benchmark(evaluation_measures)


Adult Model Initiate
Structural Equation Finished
Noise Distribution Finished
Finished Loading
Remove Prefix
End Prefix
Remove Prefix
End Prefix
Remove Prefix
End Prefix
Remove Prefix
End Prefix
Remove Prefix
End Prefix
Remove Prefix
End Prefix
Remove Prefix
End Prefix
Remove Prefix
End Prefix
Remove Prefix
End Prefix
Remove Prefix
End Prefix
Remove Prefix
End Prefix
Remove Prefix
End Prefix
Remove Prefix
End Prefix
Remove Prefix
End Prefix
Remove Prefix
End Prefix
Remove Prefix
End Prefix
Remove Prefix
End Prefix
Remove Prefix
End Prefix
Remove Prefix
End Prefix
Remove Prefix
End Prefix
Remove Prefix
End Prefix
Remove Prefix
End Prefix
Remove Prefix
End Prefix
Remove Prefix
End Prefix
Remove Prefix
End Prefix
Remove Prefix
End Prefix
Remove Prefix
End Prefix
0


'Histogram Features / true Positive / false Positive'